In [1]:
#Libraries
import pandas as pd
import numpy as np
#from datetime import dt
import math
import os

# Import code developed and functions
from feature_extraction import preprocess, flatten, label, select_group

# Ignore warnings of appending dataframes
import warnings
warnings.simplefilter('ignore')

In [5]:
path = 'C:\\Users\\mverd\\Desktop\\IMD\\ESSEX\\PredictingEmpathy\\rawdata\\'
fname = 'test\\Participant0001.tsv' 
# path = ''
# fname = 'C://Users/Ana/Desktop/Participant0001.tsv'

In [8]:
df_table = pd.read_table(path + fname, sep='\t',low_memory=False)

# Remove calibration points in recording
startPoints = df_table[df_table['Event']=='ImageStimulusStart'].index.values.astype(int)
endPoints = df_table[df_table['Event']=='ImageStimulusEnd'].index.values.astype(int)

# Store only image stimulus
df = pd.DataFrame()

for i in range(len(startPoints)):
    start = startPoints[i]
    end = endPoints[i]

    trial = df_table.iloc[start:end+1]
    df = pd.concat([df,trial])

# Columns we are keeping
df_col = ['Recording timestamp','Participant name',
            'Recording name','Recording duration',
            'Pupil diameter left','Pupil diameter right',
            'Gaze point X (MCSnorm)','Gaze point Y (MCSnorm)',
            'Eye movement type','Gaze event duration',
            'Fixation point X (MCSnorm)','Fixation point Y (MCSnorm)']

# Removing columns
df_features = df[df_col]

# Columns that need to be changed from object to float
objColumns = ['Pupil diameter left','Pupil diameter right','Gaze point X (MCSnorm)',
                'Gaze point Y (MCSnorm)','Fixation point X (MCSnorm)','Fixation point Y (MCSnorm)']

# Change (commas) to (decimals) and convert object to float64
for feature in objColumns:
    df_features[feature] = df_features[feature].str.replace(',','.').astype(float)

df_features

,Recording timestamp,Participant name,Recording name,Recording duration,Pupil diameter left,Pupil diameter right,Gaze point X (MCSnorm),Gaze point Y (MCSnorm),Eye movement type,Gaze event duration,Fixation point X (MCSnorm),Fixation point Y (MCSnorm)
3083,23419315,Participant0001,Recording1,83579,NaN,NaN,NaN,NaN,Fixation,117.0,0.5073,0.316
3084,23426120,Participant0001,Recording1,83579,NaN,NaN,0.5011,0.3203,Fixation,117.0,0.5073,0.316
3085,23434449,Participant0001,Recording1,83579,NaN,NaN,0.5016,0.3244,Fixation,117.0,0.5073,0.316
3086,23442782,Participant0001,Recording1,83579,3.21,3.14,0.5016,0.3202,Fixation,117.0,0.5073,0.316
3087,23451119,Participant0001,Recording1,83579,NaN,NaN,0.5079,0.3145,Fixation,117.0,0.5073,0.316
...,...,...,...,...,...,...,...,...,...,...,...,...
82494,43375993,Participant0001,Recording8,43548,NaN,NaN,NaN,NaN,EyesNotFound,922.0,NaN,NaN
82495,43384336,Participant0001,Recording8,43548,NaN,NaN,NaN,NaN,EyesNotFound,922.0,NaN,NaN
82496,43393702,Participant0001,Recording8,43548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82497,43393702,Participant0001,Recording8,43548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_features['Eye movement type'].unique()

array(['Fixation', 'Saccade', 'Unclassified', 'EyesNotFound', nan],
      dtype=object)

In [14]:
df_features['Eye movement type'] = df_features['Eye movement type'].replace(("EyesNotFound",np.nan), "Unclassified")
mapFixation = {'Fixation':1, 'Saccade':0, 'Unclassified':0, 'EyesNotFound':0}

df_features['Eye movement type'] = df_features['Eye movement type'].replace(mapFixation)
df_features['Eye movement type']

3083     1
3084     1
3085     1
3086     1
3087     1
        ..
82494    0
82495    0
82496    0
82497    0
82498    0
Name: Eye movement type, Length: 71206, dtype: int64

array([1, 0], dtype=int64)